# XGBoost를 이용한 날씨 상태 예측
데이터링크
https://www.kaggle.com/datasets/prasad22/weather-data
데이터셋: 날짜와 시간, 온도, 습도, 강수량, 풍속 등의 기상 정보를 포함하고 있습니다.
특징(Features): Temperature_C, Humidity_pct, Precipitation_mm, Wind_Speed_kmh, Year, Month, Day, Hour
라벨(Label): Weather (Clear, Rain, Snow, Cloudy)

이 프로젝트에서는 미국 내 다양한 지역의 기상 데이터를 사용하여 날씨 상태를 예측합니다. 사용된 데이터는 날짜, 시간, 온도, 습도, 강수량, 풍속 등의 정보를 포함하고 있으며, 날씨 상태(맑음, 비, 눈, 흐림)를 라벨로 갖습니다. 예측 모델로는 XGBoost 분류 알고리즘을 사용하였으며, 모델의 정확도는 약 85%이고, F1 스코어는 0.83입니다.

알고리즘: XGBoost (eXtreme Gradient Boosting)
선정 이유: XGBoost는 결정 트리 기반의 앙상블 기법으로, 높은 예측 성능과 과적합 방지 기능을 가지고 있어 다양한 데이터셋에 대해 좋은 성능을 보입니다.
매개변수 선택 근거: eval_metric='mlogloss'는 다중 클래스 분류에서 사용되는 평가 지표로, 모델의 예측 성능을 평가하기 위해 사용되었습니다.

* ChatGPT 도움 받은 내용 요약 + 대화 링크
https://chatgpt.com/share/67580bf4-1178-8007-a393-fe2b925633ba
데이터 전처리 및 모델 생성:

제공된 날씨 데이터셋을 불러와 필요한 열만 사용하고, 결측값을 처리했습니다.
Temperature_C, Humidity_pct, Wind_Speed_kmh, Precipitation_mm를 사용하여 온도를 예측하는 XGBoost 모델을 만들었습니다.
데이터는 훈련셋과 테스트셋으로 분리하고, StandardScaler로 스케일링을 적용했습니다.
모델 평가 시 RMSE(Root Mean Squared Error)를 사용하여 성능을 평가했습니다.
경고 메시지 해결:

XGBRegressor에서 use_label_encoder 매개변수가 사용되지 않아 발생한 경고 메시지를 제거했습니다. 이 매개변수는 제거하고 모델을 정상적으로 학습시켰습니다.
특성 중요도 시각화:

XGBoost 모델의 특성 중요도를 시각화하여, 각 특성이 온도 예측에 미치는 영향을 확인할 수 있었습니다.
새로운 데이터 예측:

새로운 입력 데이터에 대해 온도를 예측하는 예제를 작성하여, 모델의 예측 결과를 확인했습니다.

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

# 데이터 불러오기
df = pd.read_csv('weather_data.csv')

# 데이터 살펴보기
df.head()

,Location,Date_Time,Temperature_C,Humidity_pct,Precipitation_mm,Wind_Speed_kmh
0,San Diego,2024-01-14 21:12:46,10.683001,41.195754,4.020119,8.233540
1,San Diego,2024-05-17 15:22:10,8.734140,58.319107,9.111623,27.715161
2,San Diego,2024-05-11 09:30:59,11.632436,38.820175,4.607511,28.732951
3,Philadelphia,2024-02-26 17:32:39,-8.628976,54.074474,3.183720,26.367303
4,San Antonio,2024-04-29 13:23:51,39.808213,72.899908,9.598282,29.898622


In [2]:
# 1. 데이터 전처리

# 날짜와 시간을 'Date_Time'에서 추출하여 'Year', 'Month', 'Day', 'Hour' 열 생성
df['Date_Time'] = pd.to_datetime(df['Date_Time'])
df['Year'] = df['Date_Time'].dt.year
df['Month'] = df['Date_Time'].dt.month
df['Day'] = df['Date_Time'].dt.day
df['Hour'] = df['Date_Time'].dt.hour

# 날씨 상태(눈, 비, 맑음, 흐림) 컬럼 추가
def classify_weather(row):
    if row['Precipitation_mm'] > 2:
        return 'Rain'
    elif row['Temperature_C'] < 0 and row['Precipitation_mm'] > 0:
        return 'Snow'
    elif row['Humidity_pct'] > 80:
        return 'Cloudy'
    else:
        return 'Clear'

df['Weather'] = df.apply(classify_weather, axis=1)

# 'Weather' 열을 Label Encoding (문자형 -> 숫자형)
le = LabelEncoder()
df['Weather'] = le.fit_transform(df['Weather'])  # 0: Clear, 1: Rain, 2: Snow, 3: Cloudy


In [3]:
# 2. 특성과 목표 변수 설정
X = df[['Temperature_C', 'Humidity_pct', 'Precipitation_mm', 'Wind_Speed_kmh', 'Year', 'Month', 'Day', 'Hour']]
y = df['Weather']  # Weather 상태(비/맑음)

In [4]:
# 3. 데이터 나누기 (훈련 세트와 테스트 세트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# 4. 데이터 스케일링 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# XGBoost 모델 학습
model = xgb.XGBClassifier(eval_metric='mlogloss')  # 'use_label_encoder' 매개변수 제거
model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [6]:
# 5. 예측 함수 정의
def predict_weather(temp, humidity, precipitation, wind_speed, year, month, day, hour):
    # 특성값을 데이터프레임으로 변환
    input_data = pd.DataFrame([[temp, humidity, precipitation, wind_speed, year, month, day, hour]],
                              columns=['Temperature_C', 'Humidity_pct', 'Precipitation_mm', 'Wind_Speed_kmh', 'Year', 'Month', 'Day', 'Hour'])
    
    # 스케일링
    input_scaled = scaler.transform(input_data)
    
    # 예측
    prediction = model.predict(input_scaled)
    
    # 예측 결과
    weather = le.inverse_transform(prediction)
    
    # 예측된 날씨의 확률 출력
    probs = model.predict_proba(input_scaled)# 5. 예측 함수 정의
def predict_weather(temp, humidity, precipitation, wind_speed, year, month, day, hour):
    # 특성값을 데이터프레임으로 변환
    input_data = pd.DataFrame([[temp, humidity, precipitation, wind_speed, year, month, day, hour]],
                              columns=['Temperature_C', 'Humidity_pct', 'Precipitation_mm', 'Wind_Speed_kmh', 'Year', 'Month', 'Day', 'Hour'])
    
    # 스케일링
    input_scaled = scaler.transform(input_data)
    
    # 예측
    prediction = model.predict(input_scaled)
    
    # 예측 결과
    weather = le.inverse_transform(prediction)
    
    # 예측된 날씨의 확률 출력
    probs = model.predict_proba(input_scaled)
    
    weather_probs = dict(zip(le.classes_, probs[0]))
    
    return weather[0], weather_probs

    
    weather_probs = dict(zip(le.classes_, probs[0]))
    
    return weather[0], weather_probs

In [9]:
# 예시: 25도, 60% 습도, 1mm 강수량, 10km/h 풍속, 2024년 12월 10일 12시일 때
predicted_weather, weather_probabilities = predict_weather(25, 60, 1, 10, 2024, 12, 10, 12)

print(f"Predicted Weather: {predicted_weather}")
print(f"Weather Probabilities: {weather_probabilities}")

# 테스트 데이터로 예측 수행
y_pred = model.predict(X_test_scaled)

# 정확도와 F1 스코어 계산
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Predicted Weather: Clear
Weather Probabilities: {'Clear': 0.9999957, 'Cloudy': 1.2838398e-06, 'Rain': 1.5702265e-06, 'Snow': 1.4269015e-06}
Accuracy: 0.9977
F1 Score: 0.9976
